# <a id='наверх'>Обработка исходного датасета</a>

#### [1. Загрузка библиотек](#Загрузка-библиотек)
#### [2. Загрузка датасета](#Загрузка-датасета)
#### [3. Обработка пропусков](#Обработка-пропусков)
#### [4. Создание фичей](#Создание-фичей)
#### [5. Удаление клиентов ТОЛЬКО с возвратами и без покупок](#Удаление-клиентов)
#### [6. Сохранение датасета](#Сохранение-датасета)

## <a id='Загрузка-библиотек'>Загрузка библиотек</a> 
###### [наверх](#наверх)

In [3]:
import pandas as pd
import datetime as dt
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', None)

## <a id='Загрузка-датасета'>Загрузка датасета</a>
###### [наверх](#наверх)

In [4]:
# Загрузим исходный датасет
trans_df = pd.read_csv('transactions.csv')
trans_df.head(3)

,id,purch_date,channel,category_name,category,description,item,price,quantity,turnover,packing_size,uom
0,A6o1lNF4QoWbupw44vxNsw,2019-01-02,OFFLINE,"[""мешки строительные до 50 кг"", ""мешки для строительного мусора зеленые"", ""мешки для строительного мусора черные"", ""мешки для строительного мусора полипропиленовые"", ""мешки для строительного мусора пластиковые"", ""мешки строительные"", ""снижение цен на стройматериалы техническая"", ""стройматериалы"", ""строительные расходные материалы""]",др. строит. оборуд-е,мешок для строительного мусора 70 л 55x95 см полипропилен 85 мкм зеленый,10690746,9.0,2.47,22.23,NaN,NaN
1,C682dS8JSpaKaYlr5nqjhQ,2019-01-02,OFFLINE,"[""удлинители сетевые"", ""удлинители и тройники"", ""удлинитель 3 м"", ""электротовары""]",удлинители,удлинитель без заземления для маломощной техники 3 розетки 3 м цвет белый,18373471,183.0,2.47,452.01,NaN,NaN
2,G_tZ47JVT6ukGyuQYJyPRw,2019-01-02,OFFLINE,"[""семена цветов"", ""семена для выращивания цветов"", ""садок"", ""семена левкоя"", ""семена""]",семена,маттиола летний вечер» бп,18580648,1.0,14.80,14.80,NaN,NaN


In [5]:
trans_df.columns

Index(['id', 'purch_date', 'channel', 'category_name', 'category',
       'description', 'item', 'price', 'quantity', 'turnover', 'packing_size',
       'uom'],
      dtype='object')

#### Наименование колонок и их значение

* 'id' - идентификатор клиента,
* 'purch_date' - дата чека, 
* 'channel' - канал покупки (ONLINE, OFFLINE), 
* 'category_name' - список категорий, к которым относится товар, 
* 'category' - категория товара (в иерархии 2 снизу),
* 'description' - наименование товара, 
* 'item' - артикул товара, 
* 'price' - цена товара, 
* 'quantity' - количество товара в чеке, 
* 'turnover' - товарооборот по линии чека (цена * количество), 
* 'packing_size' - размер упаковки товара в условных единицах измерения для данного товара,
* 'uom' - условная (альтернативная) единица измерения товара в упаковке

In [6]:
# Максимальная дата чека в датасете
max_purch_date_of_DF = trans_df.purch_date.max()
max_purch_date_of_DF

'2021-12-31'

## <a id='Обработка-пропусков'>Обработка пропусков</a>
###### [наверх](#наверх)

In [7]:
# Отработаем пропуски
trans_df['category_name'].fillna('Unknown', inplace=True)
trans_df['packing_size'].fillna(1, inplace=True)
trans_df['uom'].fillna('EA', inplace=True)

## <a id='Создание-фичей'>Создание фичей</a>
###### [наверх](#наверх)

In [8]:
# Уникальный номер чека (ИД клиента + Дата чека + Канал покупки)
trans_df['id_check_unique'] = trans_df['id'] + trans_df['purch_date'] + trans_df['channel']

# Флаг покупки 1 = покупка, 0 = возврат
trans_df['is_purchase'] = np.where(trans_df['turnover'] > 0, 1, 0)

# Цена за единицу товара
trans_df['price_uom'] = trans_df['price'] / trans_df['packing_size']

# Количество купленных единиц товара
trans_df['quantity_uom'] = trans_df['quantity'] * trans_df['packing_size']

##### Создадим фичи на основании анализа изменчивости цен товаров

In [9]:
# Создадим датасет с ценами товаров, в каждый день их покупки
price_df = trans_df[['category_name', 'item', 'purch_date', 'price']].drop_duplicates()

In [10]:
# Данные о категории цены на основе категорий товара
def getParameterCategoryName(arg):
    if ("выгодная" in arg)|("ликвидация" in arg):
        result = 'цена понизилась'
    else:
        result = 'цена без изменений'
    return result

In [11]:
# % изменения цены
price_df['price_%'] = (price_df['price'] / price_df.groupby('item')['price'].shift(1)).fillna(1) 

# фича 1, основанная на анализе % изменения цены
price_df['price_cat1'] = np.where(price_df['price_%'] < 0.9, 'цена понизилась', 'цена без изменений') 

# фича 2, основанная на анализе категорий товара
price_df['price_cat2'] = price_df['category_name'].apply(lambda x: getParameterCategoryName(x))

In [12]:
# Обогатим исходный датасет данными об изменчивости цен товаров
trans_df = trans_df.merge(price_df[['item', 'purch_date', 'price', 'price_%', 'price_cat1', 'price_cat2']],
                          on=['item', 'purch_date', 'price'], 
                          how='left')

##### Создадим фичи на основе даты покупки

In [13]:
# год покупки
trans_df['purch_date_year'] = pd.to_datetime(trans_df['purch_date']).dt.year 

# месяц покупки
trans_df['purch_date_month'] = pd.to_datetime(trans_df['purch_date']).dt.month 

# номер календарной недели покупки
trans_df['purch_date_week'] = pd.to_datetime(trans_df['purch_date']).dt.isocalendar().week 

# номер дня недели покупки 1=ПН
trans_df['purch_date_day'] = pd.to_datetime(trans_df['purch_date']).dt.isocalendar().day 

# будний или выходной - день покупки
trans_df['purch_date_day_type'] = trans_df['purch_date_day'].apply(lambda x: 'будний' if x <= 4 else 'выходной') 

##### Создадим фичи на основе сезонности и объема продаж

In [14]:
# Данные о сезонности
def getParameterSeasonQuantile(arg, Q):
    if arg <= Q[0.25]:
        result = 1
    elif arg <= Q[0.5]:
        result = 2
    elif arg <= Q[0.75]:
        result = 3
    else:
        result = 4
    return result

In [15]:
# кол-во проданных штук в категории в текущем году
trans_df['purch_quantity_category_year'] = trans_df.groupby(
    ['category', 'purch_date_year'])['quantity'].transform(sum) 

# кол-во проданных штук в категории на текущей неделе
trans_df['purch_quantity_category_week'] = trans_df.groupby(
    ['category', 'purch_date_year', 'purch_date_week'])['quantity'].transform(sum) 

# сезонность, доля проданных штук в категории за неделю к году
trans_df['purch_season_share'] = trans_df['purch_quantity_category_week'] / trans_df['purch_quantity_category_year'] 

# Создадим словарь со значениями долей проданных штук в разных квантилях
purch_season_quantile = trans_df['purch_season_share'].quantile([0.25, 0.5, 0.75]).to_dict()

# Фича - квартили по сезонности
trans_df['purch_season_type'] = trans_df['purch_season_share'].apply(
    lambda x: getParameterSeasonQuantile(x, purch_season_quantile)) 

##### Создадим фичи на основе квартиля цены

In [16]:
# Данные о ценовых квартилях
def getParameterPriceQuantile(arg, cat, Q):
    if arg <= Q[(cat, 0.25)]:
        result = 1
    elif arg <= Q[(cat, 0.5)]:
        result = 2
    elif arg <= Q[(cat, 0.75)]:
        result = 3
    else:
        result = 4
    return result

In [17]:
# Создадим словарь со значениями алтенативной цены для разных квантилей для каждой категории
price_type_quantile = trans_df.groupby(['category'])['price_uom'].quantile([0.25, 0.5, 0.75]).to_dict()

# Фича - квартили по альтернативной цене
trans_df['price_type'] = trans_df.apply(
    lambda x: getParameterPriceQuantile(x.price_uom, x.category, price_type_quantile), axis=1) 

In [18]:
# Посчитаем средние значения объемов продаж и цен для разных вариантов значений категорий изменчивости цен на товар
df_Q1 = trans_df[(trans_df.price_cat1 == 'цена без изменений')
                   & (trans_df.price_cat2 == 'цена без изменений')][[
    'category', 'item', 'purch_date_year', 'purch_date_week', 'price_cat1', 'price_cat2', 'quantity_uom']].groupby(
    ['category', 'purch_date_year', 'purch_date_week'], as_index=False).agg(
    Q1 = ('quantity_uom', 'mean'))
df_Q2 = trans_df[(trans_df.price_cat1 == 'цена понизилась')
                   | (trans_df.price_cat2 == 'цена понизилась')][[
    'category', 'item', 'purch_date_year', 'purch_date_week', 'price_cat1', 'price_cat2', 'quantity_uom']].groupby(
    ['category', 'purch_date_year', 'purch_date_week'], as_index=False).agg(
    Q2 = ('quantity_uom', 'mean'))
df_P1 = trans_df[(trans_df.price_cat1 == 'цена без изменений')
                   & (trans_df.price_cat2 == 'цена без изменений')][[
    'category', 'item', 'purch_date_year', 'purch_date_week', 'price_cat1', 'price_cat2', 'price_uom']].groupby(
    ['category', 'purch_date_year', 'purch_date_week'], as_index=False).agg(
    P1 = ('price_uom', 'mean'))
df_P2 = trans_df[(trans_df.price_cat1 == 'цена понизилась')
                   | (trans_df.price_cat2 == 'цена понизилась')][[
    'category', 'item', 'purch_date_year', 'purch_date_week', 'price_cat1', 'price_cat2', 'price_uom']].groupby(
    ['category', 'purch_date_year', 'purch_date_week'], as_index=False).agg(
    P2 = ('price_uom', 'mean'))

In [19]:
# Дополним датасет полученными значениями, объединим по ключу (категория + год продажи + неделя продажи)
trans_df = trans_df.merge(df_Q1, on=['category', 'purch_date_year', 'purch_date_week'], how='left').merge(
    df_Q2, on=['category', 'purch_date_year', 'purch_date_week'], how='left').merge(
    df_P1, on=['category', 'purch_date_year', 'purch_date_week'], how='left').merge(
    df_P2, on=['category', 'purch_date_year', 'purch_date_week'], how='left')

##### Создадим фичи на основе эластичности цены

In [20]:
# Данные об эластичности цены
def getParameterElasticPrice(arg):
    if arg == 0:
        result = 'neutral'
    elif arg > 0 and arg <= 1:
        result = 'non_elastic'
    else:
        result = 'elastic'
    return result

In [21]:
# Фича - эластичность цены
trans_df['E'] = abs(((trans_df['Q2'] - trans_df['Q1']) / (trans_df['P2'] - trans_df['P1'])) * ((trans_df['P2'] + trans_df['P1']) / (trans_df['Q2'] + trans_df['Q1']))).fillna(0)
trans_df['elasticity_category_week'] = trans_df['E'].apply(lambda x: getParameterElasticPrice(x))

##### Создадим "total" и "средние" фичи на основе поведения клиента

In [22]:
# ТО по клиенту за весь период
trans_df['ltv_turnover'] = trans_df.groupby('id')['turnover'].transform(sum) 

# Кол-во купленных штук по клиенту за весь период
trans_df['ltv_quantity'] = trans_df.groupby('id')['quantity'].transform(sum) 

In [23]:
# Кол-во чеков с ПОКУПКАМИ, кол-во дней с покупками по клиенту за весь период
ltv_check_count_group = trans_df[trans_df.is_purchase == 1].groupby(
    'id', as_index=False).agg(
    ltv_check_count = ('id_check_unique', 'nunique'),
    ltv_purch_date_count = ('purch_date', 'nunique'))

trans_df = trans_df.merge(ltv_check_count_group, on='id', how='left') 

In [24]:
# Кол-во уникальных товаров по клиенту за весь период
ltv_item_count_group = trans_df[trans_df.is_purchase == 1].groupby(
    'id', as_index=False).agg(
    ltv_item_count = ('item', 'nunique'))

trans_df = trans_df.merge(ltv_item_count_group, on='id', how='left') 

In [25]:
# Средний чек по клиенту
trans_df['check_av_turn'] = trans_df['ltv_turnover'] / trans_df['ltv_check_count'] 

# Среднее кол-во штук товара в чеке по клиенту
trans_df['check_av_quintity'] = trans_df['ltv_quantity'] / trans_df['ltv_check_count']

# Среднее кол-во уникальных артикулов в чеке по клиенту
trans_df['check_av_item'] = trans_df['ltv_item_count'] / trans_df['ltv_check_count'] 

In [26]:
purch_dates = trans_df.groupby(
    'id', as_index=False).agg(
    min_purch_date = ('purch_date', 'min'),
    max_purch_date = ('purch_date', 'max'))

trans_df = trans_df.merge(purch_dates, on='id', how='left')

# среднее кол-во дней с покупками в месяц по клиентам
trans_df['frequence_client_per_month'] = trans_df['ltv_purch_date_count'] / ((pd.to_datetime(trans_df['max_purch_date']) - pd.to_datetime(trans_df['min_purch_date'])).dt.days.astype(np.int32) + 1 / 30.5)  

# Кол-во дней между первой и последней датами покупок по всему датасету
trans_df['days_between_min_max_date'] = (pd.to_datetime(trans_df['max_purch_date']) - pd.to_datetime(trans_df['min_purch_date'])).dt.days.astype(np.int32)  

# кол-во дней с момента последней даты покупки до последней даты покупок в датасете
trans_df['recency_client'] = (pd.to_datetime(max_purch_date_of_DF) - pd.to_datetime(trans_df['max_purch_date'])).dt.days.astype(np.int32)  

In [27]:
# накопительное кол-во купленных штук по клиенту за весь период
trans_df['ltv_quantity_cumul'] = trans_df.groupby('id')['quantity'].cumsum() 

# накопительный ТО по клиенту за весь период
trans_df['monetary'] = trans_df.groupby('id')['turnover'].cumsum() 

# накопительно частота покупок по клиенту за весь период
trans_df['frequency'] = trans_df.groupby('id')['is_purchase'].cumsum() 

In [28]:
id_recency = trans_df[trans_df.is_purchase == 1][['id', 'purch_date']].drop_duplicates()
id_recency['prev_purch_date'] = id_recency.groupby('id')['purch_date'].shift(1).fillna(id_recency['purch_date'])
id_recency['purch_date'] = pd.to_datetime(id_recency['purch_date'])
id_recency['prev_purch_date'] = pd.to_datetime(id_recency['prev_purch_date'])
id_recency['recency'] = (id_recency['purch_date'] - id_recency['prev_purch_date']).dt.days.astype(np.int32)

# кол-во дней с момента последней даты покупки до текущей даты
trans_df['purch_date'] = pd.to_datetime(trans_df['purch_date'])
trans_df = trans_df.merge(id_recency[['id', 'purch_date', 'recency']], on=['id', 'purch_date'], how='left')  

##### Создадим фичи на основе эластичности клиента

In [29]:
# Данные об эластичности клиента
def getParameterElasticClient(pen1, pen2, pen3, X, Y, Z):
    if pen1 >= X or pen2 >= Y or pen3 >= Z:
        result = 'elastic'
    else:
        result = 'non_elastic'
    return result

In [30]:
trans_df['penetration_price_cat1'] = trans_df[trans_df.price_cat1 == 'цена понизилась'].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']
trans_df['penetration_price_cat2'] = trans_df[trans_df.price_cat2 == 'цена понизилась'].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']
trans_df['penetration_price_type'] = trans_df[trans_df.price_type == 1].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']

In [31]:
price_cat1_median = (trans_df[trans_df.price_cat1 == 'цена понизилась'].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']).median()
price_cat2_median = (trans_df[trans_df.price_cat2 == 'цена понизилась'].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']).median()
price_type_median = (trans_df[trans_df.price_type == 1].groupby('id')['turnover'].transform(sum) / trans_df['ltv_turnover']).median()

In [32]:
trans_df['check_elastic'] = trans_df.apply(lambda x: getParameterElasticClient(
    x.penetration_price_cat1, x.penetration_price_cat2, x.penetration_price_type, price_cat1_median, price_cat2_median, price_type_median), axis=1) 

In [33]:
# Фича - эластичность клиента
trans_df['elasticity_client'] = trans_df.groupby('id')['check_elastic'].transform(min) 

## <a id='Удаление-клиентов'>Удаление клиентов ТОЛЬКО с возвратами и без покупок</a>
###### [наверх](#наверх)

In [34]:
# Удалим клиентов с пропусками, т.к. они только с возвратами и без покупок за всю историю
id_for_del = trans_df[trans_df.ltv_check_count.isna()]['id'].unique()

In [35]:
trans_df = trans_df[~(trans_df.id.isin(id_for_del))]

## <a id='Сохранение-датасета'>Сохранение датасета</a>
###### [наверх](#наверх)

In [36]:
trans_df[['id', 'purch_date', 'channel', 'category_name', 'category',
          'description', 'item', 'price', 'quantity', 'turnover', 'packing_size',
          'uom', 'id_check_unique', 'is_purchase', 'price_uom', 'quantity_uom',
          'price_%', 'price_cat1', 'price_cat2', 'purch_date_year',
          'purch_date_month', 'purch_date_week', 'purch_date_day',
          'purch_date_day_type', 'purch_quantity_category_year',
          'purch_quantity_category_week', 'purch_season_share',
          'purch_season_type', 'price_type', 'elasticity_category_week', 
          'ltv_turnover', 'ltv_quantity', 'ltv_check_count', 'ltv_purch_date_count', 
          'ltv_item_count', 'check_av_turn', 'check_av_quintity', 'check_av_item', 
          'frequence_client_per_month', 'days_between_min_max_date', 
          'recency_client', 'ltv_quantity_cumul', 
          'monetary', 'frequency', 'recency', 'elasticity_client']].to_pickle('transaction_and_features.pkl')